# Matrix Multiplication

In [1]:
import os


In [2]:
import ttnn

device_id = 0
device = ttnn.open_device(device_id=device_id)

2025-06-24 06:01:31.716 | DEBUG    | ttnn:<module>:83 - Initial ttnn.CONFIG:
Config{cache_path=/home/maxim_artemov/.cache/ttnn,model_cache_path=/home/maxim_artemov/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_should_raise_exception=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}


[2025-06-24 06:01:32.308] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-24 06:01:32.309] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-24 06:01:32.322] [info] [tt_cluster.cpp:177] [Device] Opening user mode device driver
[2025-06-24 06:01:32.322] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-24 06:01:32.323] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-24 06:01:32.327] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-24 06:01:32.328] [info] [pci_device.cpp:191] [SiliconDriver] Opened PCI device 0; KMD version: 2.0.0, IOMMU: disabled
[2025-06-24 06:01:32.332] [info] [cluster.cpp:277] [SiliconDriver] Harvesting mask for chip 0 is 0x40 (physical layout: 0x40, logical: 0x40, si

## Enable program cache

Enabling the program cache will speed up the execution of operations that run repeatedly

In [3]:
device.enable_program_cache()

[2025-06-24 06:01:33.605] [info] [mesh_device.cpp:518] [Metal] Enabling program cache on MeshDevice 1


# Configuration

In [4]:
m = 1024
k = 1024
n = 1024

## Initialize tensors a and b with random values

In [5]:
a = ttnn.rand((m, k), dtype=ttnn.bfloat16, device=device, layout=ttnn.TILE_LAYOUT)
b = ttnn.rand((k, n), dtype=ttnn.bfloat16, device=device, layout=ttnn.TILE_LAYOUT)

## Matrix multiply tensor a and b
The operation will run longer the first time because the kernels need to get compiled

In [6]:
output = a @ b

Re-running the operation shows significant speed up by utilizing program caching

In [7]:
output = a @ b

## Inspect the layout of matrix multiplication output

In [8]:
print(output.layout)

Layout.TILE


As can be seen, matrix multiplication produces outputs in a tile layout. That is because it's much more efficient to use this layout for computing matrix multiplications on Tenstorrent accelerators compared to a row-major layout.

And this is aslo why the logs show 2 tilize operations, as the inputs get automatically convered to the tile layout if they are in a row-major layout.

Learn more about tile layout here: TODO

## Inspect the result of the matrix multiplication

To inspect the results we will first convert to row-major layout.

In [9]:
output = ttnn.to_layout(output, ttnn.ROW_MAJOR_LAYOUT)

print("Printing ttnn tensor")
print(f"shape: {output.shape}")
print(f"chunk of a tensor:\n{output[:1, :32]}")

Printing ttnn tensor
shape: Shape([1024, 1024])
chunk of a tensor:
ttnn.Tensor([[258.00000, 256.00000,  ..., 250.00000, 256.00000]], shape=Shape([1, 32]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)


## Matrix multiply tensor a and b by using more performant config
By default, matrix multiplication might not be as effecient as it could be. To speed it up further, the user can specify how many cores they want matrix multiplication to use. This can speed up the operation significantly.

In [10]:
a = ttnn.rand((m, k), dtype=ttnn.bfloat16, device=device, layout=ttnn.TILE_LAYOUT)
b = ttnn.rand((k, n), dtype=ttnn.bfloat16, device=device, layout=ttnn.TILE_LAYOUT)

a = ttnn.to_device(a, device, memory_config=ttnn.L1_MEMORY_CONFIG)
b = ttnn.to_device(b, device, memory_config=ttnn.L1_MEMORY_CONFIG)

a = ttnn.to_layout(a, ttnn.TILE_LAYOUT)
b = ttnn.to_layout(b, ttnn.TILE_LAYOUT)

Run once to compile the kernels

In [11]:
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=ttnn.CoreGrid(y=8, x=8))

Enjoy a massive speed up on the subsequent runs

In [12]:
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=ttnn.CoreGrid(y=8, x=8))

## Close the device

In [13]:
ttnn.close_device(device)

[2025-06-24 06:01:35.334] [info] [device.cpp:1093] [Metal] Closing device 0
[2025-06-24 06:01:35.334] [info] [device.cpp:1440] [Metal] Disabling and clearing program cache on device 0
